# Assignment 4
## Description
In this assignment you must read in a file of metropolitan regions and associated sports teams from [assets/wikipedia_data.html](assets/wikipedia_data.html) and answer some questions about each metropolitan region. Each of these regions may have one or more teams from the "Big 4": NFL (football, in [assets/nfl.csv](assets/nfl.csv)), MLB (baseball, in [assets/mlb.csv](assets/mlb.csv)), NBA (basketball, in [assets/nba.csv](assets/nba.csv) or NHL (hockey, in [assets/nhl.csv](assets/nhl.csv)). Please keep in mind that all questions are from the perspective of the metropolitan region, and that this file is the "source of authority" for the location of a given sports team. Thus teams which are commonly known by a different area (e.g. "Oakland Raiders") need to be mapped into the metropolitan region given (e.g. San Francisco Bay Area). This will require some human data understanding outside of the data you've been given (e.g. you will have to hand-code some names, and might need to google to find out where teams are)!

For each sport I would like you to answer the question: **what is the win/loss ratio's correlation with the population of the city it is in?** Win/Loss ratio refers to the number of wins over the number of wins plus the number of losses. Remember that to calculate the correlation with [`pearsonr`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.pearsonr.html), so you are going to send in two ordered lists of values, the populations from the wikipedia_data.html file and the win/loss ratio for a given sport in the same order. Average the win/loss ratios for those cities which have multiple teams of a single sport. Each sport is worth an equal amount in this assignment (20%\*4=80%) of the grade for this assignment. You should only use data **from year 2018** for your analysis -- this is important!

## Notes

1. Do not include data about the MLS or CFL in any of the work you are doing, we're only interested in the Big 4 in this assignment.
2. I highly suggest that you first tackle the four correlation questions in order, as they are all similar and worth the majority of grades for this assignment. This is by design!
3. It's fair game to talk with peers about high level strategy as well as the relationship between metropolitan areas and sports teams. However, do not post code solving aspects of the assignment (including such as dictionaries mapping areas to teams, or regexes which will clean up names).
4. There may be more teams than the assert statements test, remember to collapse multiple teams in one city into a single value!

## Question 1
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NHL** using **2018** data.

In [17]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

nhl_df=pd.read_csv("assets/nhl.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

def nhl_dataframe():
    # YOUR CODE HERE
    cities_df = cities[['Metropolitan area','Population (2016 est.)[8]','NHL']].rename(columns={'Population (2016 est.)[8]':'Population'})

    #filters
    nhl_df_2018 = nhl_df[(nhl_df['year']==2018) & (nhl_df['L'].apply(lambda x: x.isnumeric()))][['team','W','L']].reset_index(drop=True)
    #correcting team names
    nhl_df_2018['team'] = nhl_df_2018['team'].apply(lambda x: re.findall('.*[^\*]',x)[0])
    #string to number on the wins and losses
    nhl_df_2018['W']=nhl_df_2018['W'].apply(lambda x: int(x))
    nhl_df_2018['L']=nhl_df_2018['L'].apply(lambda x: int(x))
    #calculating the Metropolitan area
    nhl_df_2018['Ratio'] = nhl_df_2018['W']/(nhl_df_2018['W']+nhl_df_2018['L'])
    #setting the regions
    nhl_df_2018['Metropolitan area'] = nhl_df_2018['team'].str.rpartition(' ')[0]

    #correcting some names
    nhl_df_2018['Metropolitan area'].replace({'New York':'New York City'
                                              ,'Anaheim':'Los Angeles'
                                              ,'Detroit Red':'Detroit'
                                              ,'Toronto Maple':'Toronto'
                                              ,'Arizona':'Phoenix'
                                              ,'Carolina':'Raleigh'
                                              ,'Colorado':'Denver'
                                              ,'Columbus Blue':'Columbus'
                                              ,'Dallas':'Dallas–Fort Worth'
                                              ,'Florida':'Miami–Fort Lauderdale'
                                              ,'Minnesota':'Minneapolis–Saint Paul'
                                              ,'New Jersey':'New York City'
                                              ,'San Jose':'San Francisco Bay Area'
                                              ,'Tampa Bay':'Tampa Bay Area'
                                              ,'Vegas Golden':'Las Vegas'
                                              ,'Washington':'Washington, D.C.'} , inplace=True)

    #grouping
    nhl_grouped = nhl_df_2018[['Metropolitan area','Ratio']].groupby(by='Metropolitan area').mean()

    #merge
    nhl_merged = nhl_grouped.merge(cities_df,on='Metropolitan area',how='left')
    
    return nhl_merged

def nhl_correlation(): 
    
    nhl_merged = nhl_dataframe()
    # pass in metropolitan area population from cities
    population_by_region = nhl_merged['Population'].apply(lambda x: int(x))
    # pass in win/loss ratio from nhl_df in the same order as cities["Metropolitan area"]
    win_loss_by_region = nhl_merged['Ratio']

    #raise NotImplementedError()

    assert len(population_by_region) == len(win_loss_by_region), "Q1: Your lists must be the same length"
    assert len(population_by_region) == 28, "Q1: There should be 28 teams being analysed for NHL"
    
    return stats.pearsonr(population_by_region, win_loss_by_region)[0]

In [18]:
print(f"The NHL correlation is {nhl_correlation()}")
nhl_dataframe()

The NHL correlation is 0.012486162921209895


,Metropolitan area,Ratio,Population,NHL
0,Boston,0.714286,4794447,Bruins
1,Buffalo,0.357143,1132804,Sabres
2,Calgary,0.513889,1392609,Flames
3,Chicago,0.458333,9512999,Blackhawks
4,Columbus,0.600000,2041520,Blue Jackets
5,Dallas–Fort Worth,0.567568,7233323,Stars
6,Denver,0.589041,2853077,Avalanche[note 18]
7,Detroit,0.434783,4297617,Red Wings
8,Edmonton,0.473684,1321426,Oilers
9,Las Vegas,0.680000,2155664,Golden Knights


## Question 2
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NBA** using **2018** data.

In [19]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

nba_df=pd.read_csv("assets/nba.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

def nba_dataframe():
    cities_df = cities[['Metropolitan area','Population (2016 est.)[8]','NBA']].rename(columns={'Population (2016 est.)[8]':'Population'})

    #filters
    nba_df_2018 = nba_df[(nba_df['year']==2018) & (nba_df['L'].apply(lambda x: x.isnumeric()))][['team','W','L']].reset_index(drop=True)
    #correcting team names
    nba_df_2018['team'] = nba_df_2018['team'].apply(lambda x: re.findall('[\w ]*',x)[0])
    #string to number on the wins and losses
    nba_df_2018['W']=nba_df_2018['W'].apply(lambda x: int(x))
    nba_df_2018['L']=nba_df_2018['L'].apply(lambda x: int(x))
    #calculating the Metropolitan area
    nba_df_2018['Ratio'] = nba_df_2018['W']/(nba_df_2018['W']+nba_df_2018['L'])
    #setting the regions
    nba_df_2018['Metropolitan area'] = nba_df_2018['team'].str.rpartition(' ')[0]

    #correcting some names
    nba_df_2018['Metropolitan area'].replace({'New York':'New York City'
                                              ,'Brooklyn':'New York City'
                                              ,'Anaheim':'Los Angeles'
                                              ,'Arizona':'Phoenix'
                                              ,'Carolina':'Raleigh'
                                              ,'Colorado':'Denver'
                                              ,'Dallas':'Dallas–Fort Worth'
                                              ,'Florida':'Miami–Fort Lauderdale'
                                              ,'Miami':'Miami–Fort Lauderdale'
                                              ,'Minnesota':'Minneapolis–Saint Paul'
                                              ,'New Jersey':'New York City'
                                              ,'San Jose':'San Francisco Bay Area'
                                              ,'Golden State':'San Francisco Bay Area'
                                              ,'Tampa Bay':'Tampa Bay Area'
                                              ,'Vegas Golden':'Las Vegas'
                                              ,'Washington':'Washington, D.C.'
                                              ,'Indiana':'Indianapolis'
                                              ,'Portland Trail':'Portland'
                                              ,'Utah':'Salt Lake City'} , inplace=True)

    #grouping
    nba_grouped = nba_df_2018[['Metropolitan area','Ratio']].groupby(by='Metropolitan area').mean()

    #merge
    nba_merged = nba_grouped.merge(cities_df,on='Metropolitan area',how='left')
    return nba_merged

def nba_correlation():
    # YOUR CODE HERE
    
    nba_merged = nba_dataframe()
    # pass in metropolitan area population from cities
    population_by_region = nba_merged['Population'].apply(lambda x: int(x))
    # pass in win/loss ratio from nba_df in the same order as cities["Metropolitan area"]
    win_loss_by_region = nba_merged['Ratio']
    
    #raise NotImplementedError()

    assert len(population_by_region) == len(win_loss_by_region), "Q2: Your lists must be the same length"
    assert len(population_by_region) == 28, "Q2: There should be 28 teams being analysed for NBA"

    return stats.pearsonr(population_by_region, win_loss_by_region)[0]

In [20]:
print(f"The NBA correlation is {nba_correlation()}")
nba_dataframe()

The NBA correlation is -0.17657160252844611


,Metropolitan area,Ratio,Population,NBA
0,Atlanta,0.292683,5789700,Hawks
1,Boston,0.670732,4794447,Celtics
2,Charlotte,0.439024,2474314,Hornets[note 49]
3,Chicago,0.329268,9512999,Bulls[note 9]
4,Cleveland,0.609756,2055612,Cavaliers[note 31]
5,Dallas–Fort Worth,0.292683,7233323,Mavericks
6,Denver,0.560976,2853077,Nuggets[note 17]
7,Detroit,0.475610,4297617,Pistons[note 21]
8,Houston,0.792683,6772470,Rockets
9,Indianapolis,0.585366,2004230,Pacers[note 51]


## Question 3
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **MLB** using **2018** data.

In [21]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

mlb_df=pd.read_csv("assets/mlb.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

def mlb_dataframe():
    cities_df = cities[['Metropolitan area','Population (2016 est.)[8]','MLB']].rename(columns={'Population (2016 est.)[8]':'Population'})

    #filters
    mlb_df_2018 = mlb_df[mlb_df['year']==2018][['team','W','L']].reset_index(drop=True)
    #correcting team names
    mlb_df_2018['team'] = mlb_df_2018['team'].apply(lambda x: re.findall('[\w. ]*',x)[0])
    #string to number on the wins and losses
    mlb_df_2018['W']=mlb_df_2018['W'].apply(lambda x: int(x))
    mlb_df_2018['L']=mlb_df_2018['L'].apply(lambda x: int(x))
    #calculating the Metropolitan area
    mlb_df_2018['Ratio'] = mlb_df_2018['W']/(mlb_df_2018['W']+mlb_df_2018['L'])
    #setting the regions
    mlb_df_2018['Metropolitan area'] = mlb_df_2018['team'].str.rpartition(' ')[0]

    #correcting some names
    mlb_df_2018['Metropolitan area'].replace({'New York':'New York City'
                                                ,'Brooklyn':'New York City'
                                                ,'Anaheim':'Los Angeles'
                                                ,'Arizona':'Phoenix'
                                                ,'Carolina':'Raleigh'
                                                ,'Colorado':'Denver'
                                                ,'Dallas':'Dallas–Fort Worth'
                                                ,'Texas':'Dallas–Fort Worth'
                                                ,'Florida':'Miami–Fort Lauderdale'
                                                ,'Miami':'Miami–Fort Lauderdale'
                                                ,'Minnesota':'Minneapolis–Saint Paul'
                                                ,'New Jersey':'New York City'
                                                ,'San Jose':'San Francisco Bay Area'
                                                ,'Golden State':'San Francisco Bay Area'
                                                ,'Oakland':'San Francisco Bay Area'
                                                ,'San Francisco':'San Francisco Bay Area'
                                                ,'Tampa Bay':'Tampa Bay Area'
                                                ,'Vegas Golden':'Las Vegas'
                                                ,'Washington':'Washington, D.C.'
                                                ,'Indiana':'Indianapolis'
                                                ,'Portland Trail':'Portland'
                                                ,'Utah':'Salt Lake City'
                                                ,'Boston Red':'Boston'
                                                ,'Chicago White':'Chicago'
                                                ,'Toronto Blue':'Toronto'} , inplace=True)

    #grouping
    mlb_grouped = mlb_df_2018[['Metropolitan area','Ratio']].groupby(by='Metropolitan area').mean()

    #merge
    mlb_merged = mlb_grouped.merge(cities_df,on='Metropolitan area',how='left')
    
    return mlb_merged

def mlb_correlation(): 
    # YOUR CODE HERE
    
    mlb_merged = mlb_dataframe()
    # pass in metropolitan area population from cities
    population_by_region = mlb_merged['Population'].apply(lambda x: int(x))
    # pass in win/loss ratio from mlb_df in the same order as cities["Metropolitan area"]
    win_loss_by_region = mlb_merged['Ratio']
    #raise NotImplementedError()
    
    assert len(population_by_region) == len(win_loss_by_region), "Q3: Your lists must be the same length"
    assert len(population_by_region) == 26, "Q3: There should be 26 teams being analysed for MLB"

    return stats.pearsonr(population_by_region, win_loss_by_region)[0]

In [22]:
print(f"The MLB correlation is {mlb_correlation()}")
mlb_dataframe()

The MLB correlation is 0.15027698302669307


,Metropolitan area,Ratio,Population,MLB
0,Atlanta,0.555556,5789700,Braves
1,Baltimore,0.290123,2798886,Orioles[note 46]
2,Boston,0.666667,4794447,Red Sox[note 15]
3,Chicago,0.482769,9512999,CubsWhite Sox
4,Cincinnati,0.413580,2165139,Reds[note 35]
5,Cleveland,0.561728,2055612,Indians[note 30]
6,Dallas–Fort Worth,0.413580,7233323,Rangers
7,Denver,0.558282,2853077,Rockies
8,Detroit,0.395062,4297617,Tigers[note 20]
9,Houston,0.635802,6772470,Astros


## Question 4
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NFL** using **2018** data.

In [23]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

nfl_df=pd.read_csv("assets/nfl.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

def nfl_dataframe():
     # YOUR CODE HERE
    cities_df = cities[['Metropolitan area','Population (2016 est.)[8]','NFL']].rename(columns={'Population (2016 est.)[8]':'Population'})

    #filters
    nfl_df_2018 = nfl_df[(nfl_df['year']==2018) & (nfl_df['L'].apply(lambda x: x.isnumeric()))][['team','W','L']].reset_index(drop=True)
    #correcting team names
    nfl_df_2018['team'] = nfl_df_2018['team'].apply(lambda x: re.findall('[\w. ]*',x)[0])
    #string to number on the wins and losses
    nfl_df_2018['W']=nfl_df_2018['W'].apply(lambda x: int(x))
    nfl_df_2018['L']=nfl_df_2018['L'].apply(lambda x: int(x))
    #calculating the Metropolitan area
    nfl_df_2018['Ratio'] = nfl_df_2018['W']/(nfl_df_2018['W']+nfl_df_2018['L'])
    #setting the regions
    nfl_df_2018['Metropolitan area'] = nfl_df_2018['team'].str.rpartition(' ')[0]

    #correcting some names
    nfl_df_2018['Metropolitan area'].replace({'New York':'New York City'
                                                ,'Brooklyn':'New York City'
                                                ,'Anaheim':'Los Angeles'
                                                ,'Arizona':'Phoenix'
                                                ,'Carolina':'Raleigh'
                                                ,'Colorado':'Denver'
                                                ,'Dallas':'Dallas–Fort Worth'
                                                ,'Texas':'Dallas–Fort Worth'
                                                ,'Florida':'Miami–Fort Lauderdale'
                                                ,'Miami':'Miami–Fort Lauderdale'
                                                ,'Minnesota':'Minneapolis–Saint Paul'
                                                ,'New Jersey':'New York City'
                                                ,'San Jose':'San Francisco Bay Area'
                                                ,'Golden State':'San Francisco Bay Area'
                                                ,'Oakland':'San Francisco Bay Area'
                                                ,'San Francisco':'San Francisco Bay Area'
                                                ,'Tampa Bay':'Tampa Bay Area'
                                                ,'Vegas Golden':'Las Vegas'
                                                ,'Washington':'Washington, D.C.'
                                                ,'Indiana':'Indianapolis'
                                                ,'Portland Trail':'Portland'
                                                ,'Utah':'Salt Lake City'
                                                ,'New England':'Boston'
                                                ,'Tennessee':'Nashville'} , inplace=True)

    #grouping
    nfl_grouped = nfl_df_2018[['Metropolitan area','Ratio']].groupby(by='Metropolitan area').mean()

    #merge
    nfl_merged = nfl_grouped.merge(cities_df,on='Metropolitan area',how='left')

    return nfl_merged


def nfl_correlation(): 
    
    nfl_merged = nfl_dataframe()
    # pass in metropolitan area population from cities
    population_by_region = nfl_merged['Population'].apply(lambda x: int(x))
    # pass in win/loss ratio from nfl_df in the same order as cities["Metropolitan area"]
    win_loss_by_region = nfl_merged['Ratio']
    #raise NotImplementedError()
    
    assert len(population_by_region) == len(win_loss_by_region), "Q4: Your lists must be the same length"
    assert len(population_by_region) == 29, "Q4: There should be 29 teams being analysed for NFL"

    return stats.pearsonr(population_by_region, win_loss_by_region)[0]

In [24]:
print(f"The NFL correlation is {nfl_correlation()}")
nfl_dataframe()

The NFL correlation is 0.008975826138476722


,Metropolitan area,Ratio,Population,NFL
0,Atlanta,0.437500,5789700,Falcons
1,Baltimore,0.625000,2798886,Ravens[note 45]
2,Boston,0.687500,4794447,Patriots[note 14]
3,Buffalo,0.375000,1132804,Bills[note 56]
4,Chicago,0.750000,9512999,Bears[note 8]
5,Cincinnati,0.375000,2165139,Bengals
6,Cleveland,0.466667,2055612,Browns[note 29]
7,Dallas–Fort Worth,0.625000,7233323,Cowboys
8,Denver,0.375000,2853077,Broncos
9,Detroit,0.375000,4297617,Lions


## Question 5
In this question I would like you to explore the hypothesis that **given that an area has two sports teams in different sports, those teams will perform the same within their respective sports**. How I would like to see this explored is with a series of paired t-tests (so use [`ttest_rel`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.ttest_rel.html)) between all pairs of sports. Are there any sports where we can reject the null hypothesis? Again, average values where a sport has multiple teams in one region. Remember, you will only be including, for each sport, cities which have teams engaged in that sport, drop others as appropriate. This question is worth 20% of the grade for this assignment.

In [25]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

mlb_df=pd.read_csv("assets/mlb.csv")
nhl_df=pd.read_csv("assets/nhl.csv")
nba_df=pd.read_csv("assets/nba.csv")
nfl_df=pd.read_csv("assets/nfl.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

    
def sports_team_performance():
    # YOUR CODE HERE
    mlb_df_t = mlb_dataframe()[['Metropolitan area','Ratio']]
    mlb_df_t['League']='MLB'
    nhl_df_t = nhl_dataframe()[['Metropolitan area','Ratio']]
    nhl_df_t['League']='NHL'
    nba_df_t = nba_dataframe()[['Metropolitan area','Ratio']]
    nba_df_t['League']='NBA'
    nfl_df_t = nfl_dataframe()[['Metropolitan area','Ratio']]
    nfl_df_t['League']='NFL'

    full_df=pd.concat([mlb_df_t,nhl_df_t,nba_df_t,nfl_df_t])

    sports = ['NFL', 'NBA', 'NHL', 'MLB']
    p_values = pd.DataFrame({k:np.nan for k in sports}, index=sports)

    for j in p_values.columns:
        for i in p_values.index:
            array_i = full_df[full_df['League']==i]
            array_j = full_df[full_df['League']==j]
            merged_df=array_i.merge(array_j, how='inner', on='Metropolitan area',suffixes=('_i', '_j'))
            p_values.loc[i,j] = stats.ttest_rel(merged_df['Ratio_i'],merged_df['Ratio_j'])[1]
    #raise NotImplementedError()
    
    # Note: p_values is a full dataframe, so df.loc["NFL","NBA"] should be the same as df.loc["NBA","NFL"] and
    # df.loc["NFL","NFL"] should return np.nan
    
    assert abs(p_values.loc["NBA", "NHL"] - 0.02) <= 1e-2, "The NBA-NHL p-value should be around 0.02"
    assert abs(p_values.loc["MLB", "NFL"] - 0.80) <= 1e-2, "The MLB-NFL p-value should be around 0.80"
    return p_values

In [26]:
sports_team_performance()

,NFL,NBA,NHL,MLB
NFL,NaN,0.940611,0.024714,0.802069
NBA,0.940611,NaN,0.022297,0.950540
NHL,0.024714,0.022297,NaN,0.000708
MLB,0.802069,0.950540,0.000708,NaN
